In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install tiktoken
!pip install pydantic
!pip install python-dotenv

When we want to convert openai functions to pydantic objects (for a more structured approach for defining tools for LLMs) we can leverage the following ideas:

In [9]:
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function


class WeatherSearch(BaseModel):
    """Call this function to get the weather of a specific airport."""
    airport_code: str = Field(description="The airport code of the airport you want to know the weather of.")
    

openai_function_weather_search = convert_pydantic_to_openai_function(WeatherSearch)

SO the airport_code is something that the LLM will figure out on how to input to the function, therefore making the call with the correct input.

In [45]:
from langchain.chat_models import ChatOpenAI

llm_chat = ChatOpenAI()

llm_chat.invoke("What's the weather like in Lisbon today?",functions=[openai_function_weather_search])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "LIS"\n}', 'name': 'WeatherSearch'}})

We can see from below that the LLM figured out that the airport_code for Lisbon is `LIS`.

You can also bind the function to the model to avoid having to pass the `functions` keyword argument everytime. 

In [17]:
model_with_function = llm_chat.bind(functions=[openai_function_weather_search])
model_with_function.invoke("What's the weather like in Lisbon today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "LIS"\n}', 'name': 'WeatherSearch'}})

YOu can also force the model to use the function by binding it with it and adding the `function_call` keyword argument to a dictionary containing the key `name` and the name of that function `WeatherSearch`:

In [16]:
model_with_forced_function = llm_chat.bind(functions=[openai_function_weather_search],function_call={"name": "WeatherSearch"})

Now, let's connect to the stuff we already know.

In [24]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that tells the weather"),
    ("user", "{input}")
])

llm_chat = ChatOpenAI(temperature=0)

llm_chat_with_forced_function = llm_chat.bind(functions=[openai_function_weather_search], function_call={"name": "WeatherSearch"})

chain = prompt | llm_chat_with_forced_function

chain.invoke({"input": "What's the weather like in Lisbon today?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "LIS"\n}', 'name': 'WeatherSearch'}})

For multiple functions:

In [40]:
from typing import List

class SearchExamples(BaseModel):
    """Call this function to search for examples of a concept."""
    concept: str = Field(description="The concept you want to search for.")
    
class AnalogySearch(BaseModel):
    """Call this function to search for analogies of a concept."""
    concept: str = Field(description="The concept you want to search for.")


openai_function_analogy_search = convert_pydantic_to_openai_function(AnalogySearch)
openai_function_example_search = convert_pydantic_to_openai_function(SearchExamples) 

In [41]:
functions = [
    openai_function_analogy_search,
    openai_function_example_search
]

In [42]:
llm_chat_with_functions = llm_chat.bind(functions=functions)

In [43]:
# Here the model calls the example function
llm_chat_with_functions.invoke("List examples for what and SDK is.")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "concept": "SDK"\n}', 'name': 'SearchExamples'}})

In [44]:
llm_chat_with_functions.invoke("List analogies for what and SDK is.")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "concept": "SDK"\n}', 'name': 'AnalogySearch'}})